In [ ]:
from datetime import datetime
import mailbox
import re
from zoneinfo import ZoneInfo

import pandas as pd
import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.pyplot as plt


def parse_gmail_timestamp(date_str: str) -> datetime:
    """Convert Gmail UTC timestamp to Eastern Time datetime object."""
    return datetime.strptime(date_str, "%a, %d %b %Y %H:%M:%S %z").astimezone(ZoneInfo("America/New_York"))


def parse_price_from_subject(subject_line: str) -> float | None:
    """Extract price from email subject line, assume formatted as $..."""
    match = re.search(r'\$([\d\.]+)', subject_line)
    return float(match.group(1)) if match is not None else None

In [ ]:
mbox = mailbox.mbox("maize_tix_bot.mbox")

lowest_asks: list[float] = []
datetimes: list[datetime] = []

for message in reversed(mbox):
    if (ask_price := parse_price_from_subject(message["subject"])) is not None:
        lowest_asks.append(ask_price)
        datetimes.append(parse_gmail_timestamp(message["date"]))

In [ ]:
df_prices = pd.DataFrame(data={"time": datetimes, "price": lowest_asks})
df_prices = df_prices[df_prices['price'].shift() != df_prices['price']].sort_values('time').reset_index(drop=True)

In [ ]:
sns.set_style("whitegrid")
fig, ax = plt.subplots(figsize=(12, 6))

df_prices_no_outlier = df_prices[df_prices["price"] >= 60]
df_prices_no_outlier["time_num"] = mdates.date2num(df_prices_no_outlier["time"])
sns.lineplot(data=df_prices_no_outlier, x="time", y="price", ax=ax, label="Lowest Ask")
sns.regplot(data=df_prices_no_outlier, x="time_num", y="price", lowess=True, scatter=False, ax=ax, color='orange', label="LOWESS-Smoothed Curve")

sns.despine(top=True, right=True, left=True, bottom=True)
plt.legend()
plt.xlabel("Date")
plt.ylabel("Lowest Ask Price ($)")
plt.title("MaizeTix Ticket Price Trend")
plt.tight_layout()
plt.savefig("price_trend.png", dpi=300)
plt.show()